In [1]:
import logging

from dash import Dash, html, dash_table, dcc, callback, Output, Input
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

e = list()

In [2]:
errors = pd.read_csv('../RESULTS.csv').set_index('Unnamed: 0')
errors['Scaling Method'] = errors['Scaling Method'].fillna('None')
errors.columns

Index(['Technique', 'Scaling Method', 'Variables', 'Fold',
       'Explained Variance Score', 'Max Error', 'Mean Absolute Error',
       'Root Mean Squared Error', 'Median Absolute Error',
       'Mean Absolute Percentage Error', 'r2', 'Field', 'Comparison'],
      dtype='object')

In [3]:
external_stylesheets = [dbc.themes.MORPH]
app = Dash(__name__, external_stylesheets=external_stylesheets)

In [4]:
radio_options = list()
for col in ['Comparison', 'Field', 'Technique', 'Scaling Method', 'Variables']:
    print(errors[col].unique())
    radio_options.append(dbc.Col([dcc.Checklist(np.sort(errors[col].unique()), id=col)], width="auto", style={ "overflow-y":"scroll", "height": "100px"}))

@callback(
    Output('filtered_df', 'data'),
    Input('Comparison', 'value'),
    Input('Field', 'value'),
    Input('Technique', 'value'),
    Input('Scaling Method', 'value'),
    Input('Variables', 'value'),
)
def choose_data(comp, field, tech, sca, var):
    options = {
        "Comparison": comp,
        "Field": field,
        "Technique": tech,
        "Scaling Method": sca,
        "Variables": var
    }
    df = errors.copy(deep=True)

    for label, col in options.items():

        if not col:
            col_a = list(errors[label].unique())
        else:
            col_a = col

        df = df[df[label].isin(col_a)]
    return df.to_json(orient='split')

@callback(Output('error-table', 'figure'), Input('filtered_df', 'data'))
def update_table(jsonified_cleaned_data):
    df = pd.read_json(jsonified_cleaned_data, orient='split')
    fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=df.transpose().values.tolist(),
               fill_color='lavender',
               align='left'))
    ])
    return fig

@callback(Output('box-plot-rel', 'figure'), Input('filtered_df', 'data'))
def boxplot(jsonified_cleaned_data):
    df = pd.read_json(jsonified_cleaned_data, orient='split')
    fig = px.box(df, y=["r2", 'Explained Variance Score', 'Mean Absolute Percentage Error'], points="all", hover_data=['Comparison', 'Field', 'Technique', 'Scaling Method', 'Variables', 'Fold'], range_y=[0,1])

    return fig

@callback(Output('box-plot-raw', 'figure'), Input('filtered_df', 'data'))
def boxplot(jsonified_cleaned_data):
    df = pd.read_json(jsonified_cleaned_data, orient='split')
    fig = px.box(df, y=['Max Error', 'Mean Absolute Error', 'Root Mean Squared Error', 'Median Absolute Error'], points="all", hover_data=['Comparison', 'Field', 'Technique', 'Scaling Method', 'Variables', 'Fold'], range_y=[
       0, 1000
    ])

    return fig
    

['AirView Low-Cost B vs Bushy Park FDMS' 'Nova Sensor A vs Bushy Park OPC'
 'Nova Sensor B vs Bushy Park FDMS'
 'AirView Low-Cost A vs Bushy Park FDMS'
 'Nova Sensor A vs AirView Reference A'
 'AirView Reference B vs AirView Reference A'
 'Nova Sensor A vs Bushy Park FDMS'
 'AirView Low-Cost B vs AirView Reference A'
 'Nova Sensor B vs AirView Reference A'
 'AirView Reference A vs Bushy Park FDMS'
 'AirView Reference B vs Bushy Park FDMS'
 'Nova Sensor B vs Bushy Park OPC'
 'AirView Low-Cost A vs AirView Reference A']
['PM2.5' 'PM10' 'PM4' 'PM1']
['Bayesian Automatic Relevance Detection' 'Bayesian Ridge Regression'
 'Decision Tree' 'Gradient Boosting Regression' 'Linear Regression'
 'RANSAC' 'Lasso Regression' 'Support Vector Regression'
 'Lasso Regression (Cross Validated)' 'Multi-Layer Perceptron Regression'
 'Quantile Regression' 'Least Angle Lasso Regression'
 'Least Angle Regression' 'Linear Support Vector Regression'
 'Isotonic Regression' 'Passive Aggressive Regression' 'Huber R

In [5]:
app.layout = dbc.Container([
    dbc.Row([
        html.Div('Graddnodi', className="text-primary text-center fs-3")
    ]),

    dbc.Row(radio_options, justify="center", align="start"),
    dbc.Row(
        [
            dcc.Graph(figure={}, id='error-table')
        ]
    ),
    dbc.Row(
        [
            dbc.Col([dcc.Graph(figure={}, id='box-plot-rel')]),
            dbc.Col([dcc.Graph(figure={}, id='box-plot-raw')]),
        ]
    ),
    dcc.Store(id='filtered_df')

], fluid=True)

In [6]:
errors.columns

Index(['Technique', 'Scaling Method', 'Variables', 'Fold',
       'Explained Variance Score', 'Max Error', 'Mean Absolute Error',
       'Root Mean Squared Error', 'Median Absolute Error',
       'Mean Absolute Percentage Error', 'r2', 'Field', 'Comparison'],
      dtype='object')

In [7]:
app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


In [8]:
print(e)

[]
